### Anna Urbala
# Implementacja NEAT

Z racji tego, że neat jest skomplikowanym podejściem, jego implementacja rozbita jest na dwa tygodnie. W ramach pierwszego deadline należy przesłać niekoniecznie kompletny i działający kod, który będzie dowodem rozpoczęcia pracy nad rozwiązaniem. Dopiero kod w kolejnym tygodniu musi w pełni działać.

W tym tygodniu trzeba zaimplementować: kodowanie, w szczególności generowanie fenotypu z genotypu, wyliczanie funkcji dopasowania i część operatora mutacji polegająca na dodaniu neuronu i zmianie wagi.

W ramach testu przeprowadzić próbne uczenie (bez krzyżowania) na zbiorze easy z laboratorium o sieciach neuronowych.

https://github.com/gary-butler/NEAT

https://towardsdatascience.com/neuro-evolution-on-steroids-82bd14ddc2f6

https://github.com/SirBob01/NEAT-Python/blob/master/neat/neat.py

https://github.com/goktug97/NEAT/blob/master/neat/neat.py

https://github.com/vishnugh/evo-NEAT/tree/master/src/com/evo/NEAT

https://www.youtube.com/watch?v=1I1eG-WLLrY

http://nn.cs.utexas.edu/downloads/papers/stanley.cec02.pdf

In [34]:
from enum import Enum, auto
import random
from copy import deepcopy

class NodeType(Enum):
    INPUT = auto()
    HIDDEN = auto()
    OUTPUT = auto()

class ConnectionGene:
    def __init__(self, inNode: int, outNode: int, weight=1.0, expressed=True, innovation=0):
        self.inNode = inNode
        self.outNode = outNode
        self.weight = weight
        self.expressed = expressed
        self.innovation = innovation
    
    def disable(self):
        self.expressed = False

class NodeGene:
    def __init__(self, type_: NodeType, id_: int):
        self.type = type_
        self.id = id_

class Counter:
    def __init__(self):
        self.innovation = 0
        
    def getInnovation(self):
        self.innovation += 1
        return self.innovation
    
class Genome:
    PROB_PERTURBING = 0.9
    
    def __init__(self):
        self.connections = {}
        self.nodes = {}
    
    def addNodeGene(self, gene: NodeGene):
        self.nodes[gene.id] = gene
    
    def addConnectionGene(self, gene: ConnectionGene):
        self.connections[gene.innovation] = gene
        
    def mutation(self):
        for conn in self.connections.values():
            if random.uniform(0, 1.0) < self.PROB_PERTURBING:
                conn.weight *= random.uniform(-2.0, 2.0)
            else:
                conn.weight = random.uniform(-2.0, 2.0)
            
    
    def addConnectionMutation(self, innovation: Counter):
        node1 = self.nodes[random.ranrange(len(self.nodes))]
        node2 = self.nodes[random.ranrange(len(self.nodes))]
        
        rev = False
        if node1.type.value > node2.type.value:
            rev = True
            node1,node2 = node2,node1
        
        connectionExists = False
        for conn in self.connections.values():
            if (conn.inNode == node1.id and conn.outNode == node2.id) \
            or (conn.inNode == node2.id and conn.outNode == node1.id):
                connectionExists = True
                break
        
        if not connectionExists:
            newConn = ConnectionGene(node1.id, node2.id, weight=random.uniform(-1.0, 1.0), innovation=innovation.getInnovation())
            self.addConnectionGene(newConn)
    
    def addNodeMutation(self, innovation: Counter):
        conn = self.connections[random.ranrange(len(self.connections))]
        
        inNode = self.nodes[conn.inNode]
        outNode = self.nodes[conn.outNode]
        
        conn.disable()
        
        newNode = NodeGene(NodeType.HIDDEN, len(self.nodes))
        newConn1 = ConnectionGene(inNode.id, newNode.id, innovation=innovation.getInnovation())
        newConn2 = ConnectionGene(newNode.id, outNode.id, weight=conn.weight, innovation=innovation.getInnovation())
        
        self.addNodeGene(newNode)
        self.addConnectionGene(newConn1)
        self.addConnectionGene(newConn2)
    
    @staticmethod
    def crossover(parent1: Genome, parent2: Genome):
        child = Genome()
        for parent1Node in parent1.nodes.values():
            child.nodes.append(deepcopy(parent1Node))
        
        for parent1Conn in paren1.connections.values():
            childConnGene = deepcopy(parent1Conn)
            if parent1Conn.innovation in parent2.connections and random.choice([True, False]):
                childConnGene = deepcopy(parent2.connections[parent1Conn.innovation])
            child.addConnectionGene(childConnGene)
            
        return child
    
    @staticmethod
    def compatibilityDistance(genome1: Genome, genome2: Genome, c1: int, c2: int, c3: int):
        excessGenes = Genome.countExcessGenes(genome1, genome2)
        disjointGenes = Genome.countDisjointGenes(genome1, genome2)
        avgWeightDiff = Genome.averageWeightDiff(genome1, genome2)
        return excessGenes * c1 + disjointGenes * c2 + avgWeightDiff * c3
    
    @staticmethod
    def countMatchingGenes(genome1: Genome, genome2: Genome):
        matchingGenes = 0
        
        highestInnovation1 = max(genome1.nodes.keys())
        highestInnovation2 = max(genome2.nodes.keys())
        indices = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if i in genome1.nodes and i in genome2.nodes:
                matchingGenes += 1
        
        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if i in genome1.connections and i in genome2.connections:
                matchingGenes += 1
        
        return matchingGenes
    
    @staticmethod
    def countDisjointGenes(genome1: Genome, genome2: Genome):
        disjointGenes = 0
        
        highestInnovation1 = max(genome1.nodes.keys())
        highestInnovation2 = max(genome2.nodes.keys())
        indices = min(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if (i not in genome1.nodes and i in genome2.nodes) \
            or (i not in genome2.nodes and i in genome1.nodes):
                disjointGenes += 1
        
        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices = min(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if (i not in genome1.connections and i in genome2.connections) \
            or (i not in genome2.connections and i in genome1.connections):
                disjointGenes += 1
        
        return disjointGenes
    
    @staticmethod
    def countExcessGenes(genome1: Genome, genome2: Genome):
        excessGenes = 0
        
        highestInnovation1 = max(genome1.nodes.keys())
        highestInnovation2 = max(genome2.nodes.keys())
        indices1 = min(highestInnovation1, highestInnovation2)
        indices2 = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices1+1, indices2+1):
            if (i not in genome1.nodes and i in genome2.nodes) \
            or (i not in genome2.nodes and i in genome1.nodes):
                excessGenes += 1
        
        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices1 = min(highestInnovation1, highestInnovation2)
        indices2 = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices1+1, indices2+1):
            if (i not in genome1.connections and i in genome2.connections) \
            or (i not in genome2.connections and i in genome1.connections):
                excessGenes += 1
        
        return excessGenes
    
    @staticmethod
    def averageWeightDiff(genome1: Genome, genome2: Genome):
        matchingGenes = 0
        weightDifference = 0.0

        highestInnovation1 = max(genome1.connections.keys())
        highestInnovation2 = max(genome2.connections.keys())
        indices = max(highestInnovation1, highestInnovation2)
        
        for i in range(indices + 1):
            if i in genome1.connections and i in genome2.connections:
                matchingGenes += 1
                weightDifference += abs(genome1.connections[i].weight - genome2.connections[i].weight)
        
        return weightDifference/matchingGenes
    

In [30]:
a = {1: "4", 4: "3"}
6 in a

False

In [32]:
abs(-4)

4